## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= 'color: #48e0dc'>Packeges

In [42]:
import time
import random
import pickle
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from IPython.display import set_matplotlib_formats
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [48]:
%matplotlib inline
COLORS = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

ip_port_list = [('75.84.199.80', '80'),
                ('172.173.241.207', '80'),
                ('209.145.60.213', '80'),
                ('86.109.3.28', '80'),
                ('65.111.241.211', '80'),
                ('144.34.162.125', '80')]

## <center><strong>Setting up the<span style= 'color: #5cd3f7'> web scrapers

In [55]:
data_jobs_titles = ['Data entry', 'Data engineer']
                    # 'Data scientist', 'Data analyst',
                    # 'ML devoloper']

indeed_countries = ['de', 'uk']#, 'www'] USA don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']
ip_port = random.choice(ip_port_list)

In [72]:
def scrape_page(url: str, ip_and_port: tuple[str] = None, retrieve_new_url=None) -> BeautifulSoup:
    max_retries = 3
    retry_delay = 5
    
    chrome_options = Options()
    if ip_and_port:
        chrome_options.add_argument(f'--proxy-server={ip_and_port[0]}:{ip_and_port[1]}')
    driver = webdriver.Chrome(options=chrome_options)
    
    for retry in range(max_retries):
        try:
            driver.get(url)
            wait = WebDriverWait(driver, 15)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            if retrieve_new_url:
                new_url = driver.current_url
                driver.quit()
                return soup, new_url
            
            driver.quit()
            return soup
        
        except Exception as e:
            print(f"Error: {e}")
            print(f"Retrying ({retry + 1}/{max_retries}) after {retry_delay} seconds...")
            time.sleep(retry_delay)

    print("Maximum retries exceeded. Unable to scrape the page.")
    driver.quit()
    return None

In [61]:
def indeed_scraper(page: int,
                   country: str,
                   job_title: str) -> BeautifulSoup:

    modefid_page = (page - 1) * 10

    data_job_title = job_title.replace(' ', '+')

    # Pages in indeed.com are zero based indexed
    url: str = f'https://{country}.indeed.com/jobs?q={data_job_title}&sc=0kf%3Aattr%28DSQF7%29%3B&start={modefid_page}' + \
    '&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf' + \
    '6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2Dlh' + \
    'UPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea'

    soup = scrape_page(url)
    loaded_page = int(soup.find('button', 'css-ns2mzi e8ju0x51').text)

    # available_pages = []

    # for button in soup.find_all('button', 'css-1qt7hdn e8ju0x50'):
    #     available_pages.append(int(button.text))

    if (int(page) != loaded_page):
        # We will use KeyError as standard to represent getting out of the max pages.
        raise KeyError
        
    print(f'Page {page} Loaded successfully.')

    return  soup

In [62]:
def check_for_content(driver):

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    try:
        driver.find_element(By.CLASS_NAME, 'loader loader--show')
        continue_ = True

    except:
        try:
            driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button infinite-scroller__show-more-button--visible')
            continue_ = True

        except:
            continue_ = False

    return  continue_

In [63]:
def linkedin_scraper(country: str, # This scraper scrapes all pages at once
                     job_title: str) -> BeautifulSoup:

    # I know that code logic is wrong but at least it's running completly fine.

    driver = webdriver.Chrome()
    url: str = f'https://www.linkedin.com/jobs/search/?keywords={job_title}&' + \
                f'location={country}&refresh=true&start=0&position=1&pageNum=0'

    driver.get(url)

    while True:
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

            wait = WebDriverWait(driver, 10)
            wait.until(lambda driver: check_for_content(driver))
            # print(is_element_present(driver))

        except:
            break

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    return  soup

## <center><strong><span style= 'color: #4895e0'>Collecting</span> jobs data

**Now we will need for three steps to collect the data we need:**

1. Collect all pages we need from each scraper and stack them together as HTML code.
2. Scrape for jobs cards links from linkedin and indeed.
3. Scrape for the data in those links and store the data in a DataFrame then in SQLite
4. We may also consider collecting total results per each platform, country etc ..

#### **Stack pages together as HTML code**

In [64]:
soups = {'soup': [],
         'job_title': [],
         'country': [],
         'platform': []}

flag: bool = False
max_page: int = 7

for country in indeed_countries:
    for job_title in data_jobs_titles:
        stacked_pages_soup: str = ''

        for i in count(0):

            try:
                page = i + 1
                soup: BeautifulSoup = indeed_scraper(country= country,
                                                     job_title= job_title,
                                                     page= page)
                
                stacked_pages_soup += '\n<br> ' + str(soup)
                if page > max_page:
                    print('Finished loading current country or job title.\n')
                    break

            except KeyError as e:
                print('Finished loading current country or job title.\n')
                break

        soups['platform'].append('Indeed')
        soups['soup'].append(stacked_pages_soup)
        soups['job_title'].append(job_title)
        soups['country'].append(country)

for country in linkedin_countries:
    for job_title in data_jobs_titles:

        soup: BeautifulSoup = linkedin_scraper(country= country,
                                job_title= job_title)

        soups['platform'].append('LinkedIn')
        soups['soup'].append(soup)
        soups['job_title'].append(job_title)
        soups['country'].append(country)

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Finished loading current country or job title.

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Page 3 Loaded successfully.
Page 4 Loaded successfully.
Page 5 Loaded successfully.
Page 6 Loaded successfully.
Page 7 Loaded successfully.
Page 8 Loaded successfully.
Finished loading current country or job title.

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Page 3 Loaded successfully.
Page 4 Loaded successfully.
Page 5 Loaded successfully.
Page 6 Loaded successfully.
Page 7 Loaded successfully.
Page 8 Loaded successfully.
Finished loading current country or job title.

Page 1 Loaded successfully.
Page 2 Loaded successfully.
Page 3 Loaded successfully.
Page 4 Loaded successfully.
Page 5 Loaded successfully.
Page 6 Loaded successfully.
Page 7 Loaded successfully.
Page 8 Loaded successfully.
Finished loading current country or job title.



In [67]:
pd.DataFrame(soups).head(6)

,soup,job_title,country,platform
0,"\n<br> <html class=""js-focus-visible"" data-js-...",Data entry,de,Indeed
1,"\n<br> <html class=""js-focus-visible"" data-js-...",Data engineer,de,Indeed
2,"\n<br> <html class=""js-focus-visible"" data-js-...",Data entry,uk,Indeed
3,"\n<br> <html class=""js-focus-visible"" data-js-...",Data engineer,uk,Indeed
4,"[[[\n, <meta content=""d_jobs_guest_search"" nam...",Data entry,European Union,LinkedIn
5,"[[[\n, <meta content=""d_jobs_guest_search"" nam...",Data engineer,European Union,LinkedIn


#### **Collecting jobs links**

In [127]:
soups = pd.DataFrame(soups) # I will convert the soups dict to a df so we can do masking easly.

soups['total_jobs'] = np.nan
soups['jobs_links'] = np.nan
soups['companies_links'] = np.nan

indeed_jobs_links: set = set()
indeed_companies_links: set = set()

linkedin_jobs_links: set = set()
linkedin_companies_links: set = set()


# Collecting indeed jobs links
for i, soup in enumerate(soups[soups['platform'] == 'Indeed']['soup']):
    total_jobs = len(BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'))

    for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
        job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
        indeed_jobs_links.append(job_link)

    for job_link in job_links[:5]:
        print(scrape_page(job_link).find('a', 'css-775knl emf9s7v0')['href'])
    # for job_id in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0')[:5]: # TOREMOVE
    #     job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job_id['data-jk'])
    #     company_link = scrape_page(job_link).find('a', 'css-775knl emf9s7v0')['href']
        
        indeed_jobs_links.add(job_link)
        indeed_companies_links.add(company_link)

    soups['total_jobs'][i] = total_jobs
    soups['jobs_links'][i] = indeed_jobs_links
    soups['companies_links'][i] = indeed_companies_links



# Collecting linkedIn jobs links
# for i, soup in enumerate(soups[soups['platform'] == 'LinkedIn']['soup']):
    
#     soup = BeautifulSoup(soup)
#     job_card_class = 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'
#     total_jobs = len(soup.find_all('a', job_card_class))

#     for job_card in soup.find_all('a', job_card_class)[:5]: # TOREMOVE
#         job_link = job_card['href']

#         linkedin_jobs_links.add(job_link)
#         linkedin_companies_links.add(scrape_page(job_link).find('a', 'css-775knl emf9s7v0')['href'])

#     soups['total_jobs'][i] = total_jobs
#     soups['jobs_links'][i] = linkedin_jobs_links
#     soups['companies_links'][i] = linkedin_companies_links

Error: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A5A813+48355]
	(No symbol) [0x009EC4B1]
	(No symbol) [0x008F5358]
	(No symbol) [0x008DD293]
	(No symbol) [0x0093E37B]
	(No symbol) [0x0094C473]
	(No symbol) [0x0093A536]
	(No symbol) [0x009182DC]
	(No symbol) [0x009193DD]
	GetHandleVerifier [0x00CBAABD+2539405]
	GetHandleVerifier [0x00CFA78F+2800735]
	GetHandleVerifier [0x00CF456C+2775612]
	GetHandleVerifier [0x00AE51E0+616112]
	(No symbol) [0x009F5F8C]
	(No symbol) [0x009F2328]
	(No symbol) [0x009F240B]
	(No symbol) [0x009E4FF7]
	BaseThreadInitThunk [0x772600C9+25]
	RtlGetAppContainerNamedObjectPath [0x773A7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x773A7AEE+238]

Retrying (1/3) after 5 seconds...
Error: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:


AttributeError: 'NoneType' object has no attribute 'find'

In [84]:
soups

{'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=0f4e49c98c0536f1',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=19f3a3be9f35218f',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=1df95ac216434def',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=297a5f03a72d28ba',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=40e35f9a902792ed',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=446ba27dfffdbe36',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=48746eece7f6d42f',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=4be75a2883ca49b4',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=5010e97afafcfbf9',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=605f5275b7921be9',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=973410652b64f9a8',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=a34fc6b420419272',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=bbae1d266770b45a',
 'https://ww

In [117]:
for soup in soups[soups['platform'] == 'Indeed']['soup'][:2]:
    job_links = []

    for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
        job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
        job_links.append(job_link)

     for job_link in job_links[:5]:
         print(scrape_page(job_link).find('a', 'css-775knl emf9s7v0')['href'])

    print('\n')

27
120
120
120


In [115]:
len(set(job_links))

387

In [114]:
len(job_links) 

387

In [ ]:
test_html = scrape_page('https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=26c4d3754f443acb')


In [ ]:
test_html.find_all('div','jobsearch-JobMetadataFooter')